In [4]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [5]:
%matplotlib inline

from pprint import pprint
from zen_garden.postprocess.results import Results

from pathlib import Path
from os import chdir

In [6]:
# set path
path = Path('/Users/davidbertschinger/Documents/GitHub/ZEN-garden/notebooks')
chdir(path)

In [7]:
# cost optimal solution
res_adjusted = Results("../data/outputs/03_example_CCTS_N-1_03_CT_infeasible_adjusted_scenarios")
for scenario in res_adjusted.scenarios:
    res_adjusted.get_df('flow_export', scenario=scenario, to_csv="flow_export_adjusted")
    
    # get flow transport csv
    df = pd.read_csv('flow_export_adjusted.csv')
    # find unique failure states
    states = df['failure_states'].unique()
    time_steps = df['time_operation'].unique()
    # find unique technologies
    co2_stored = df[df['carrier'] == "co2_stored"]
    # Initialize an empty list to store data
    table_data = []
    
    # Iterate over time_steps
    for time_step in range(len(time_steps)):
        co2_stored_time_step = co2_stored[co2_stored['time_operation'] == time_step]
        co2_stored_no_failure = co2_stored_time_step[
            co2_stored_time_step['failure_states'] == "no_failure_technology: no_failure_location"].sum()
        co2_not_stored = (len(states) * co2_stored_no_failure["flow_export"] - co2_stored_time_step.sum()[
            "flow_export"]) * 8.76
    
        # Append data as a tuple to the list
        table_data.append((time_steps[time_step], co2_not_stored))
    
    # Create a DataFrame from the list
    table_df = pd.DataFrame(table_data, columns=["year", "carbon_emissions_limit"])
    table_df['year'] = table_df['year'] + 2025
    

In [116]:
# Initialize the Results object
res_adjusted = Results("../data/outputs/03_example_CCTS_N-1_03_CT_infeasible_adjusted_scenarios")

flow = 'flow_export'
carrier = "co2_stored"

# Initialize an empty DataFrame to store the combined results
combined_df = pd.DataFrame()

# Iterate over each scenario
for scenario in res_adjusted.scenarios:
    # Export the flowdata to CSV for the current scenario
    res_adjusted.get_df(flow, scenario=scenario, to_csv="flow_adjusted.csv")
    res_adjusted.get_df("cost_total", scenario=scenario, to_csv="opex_yearly.csv")
    
    # Load the exported data
    df = pd.read_csv('flow_adjusted.csv')
    opex = pd.read_csv('opex_yearly.csv')
    
    # Find unique failure states and time steps
    time_steps = df['time_operation'].unique()
    
    # Filter for carrier
    df = df[df['carrier'] == carrier]
    
    # Initialize a list to store data for the current scenario
    scenario_data = []

    # Iterate over time_steps
    for time_step in time_steps:
        # Filter data for the current time_step
        df_time_step = df[df['time_operation'] == time_step]
        cost_total = opex[opex['year'] == time_step]
        # Sum up the flow where there's no failure
        df_no_failure = df_time_step[df_time_step['failure_states'] == "no_failure_technology: no_failure_location"][flow].sum()
        # difference in flow for carrier fore each failure state minus no failure
        flow_difference = (df_time_step[flow].sum() - len(df['failure_states'].unique()) * df_no_failure) * 8.76
        total_flow = (df_time_step[flow].sum() - (len(df['failure_states'].unique())-1) * df_no_failure) * 8.76
        cost_per_total_flow = (cost_total['cost_total'].sum())/1000
        
        # Append the result to the list
        scenario_data.append(round(cost_per_total_flow,1))
    
    # Modify the scenario name to only include the part after the last underscore
    simplified_scenario_name = scenario.split('_')[-1] 
    # Create a DataFrame from the collected scenario data
    table_df = pd.DataFrame({"year": time_steps, simplified_scenario_name: scenario_data})
    
    # Adjust year based on your setup
    table_df['year'] = table_df['year'] + 2025
    
    if combined_df.empty:
        combined_df = table_df.set_index('year')
    else:
        # Merge the new scenario data into the combined DataFrame
        combined_df = combined_df.merge(table_df.set_index('year'), left_index=True, right_index=True, how='outer')

# After processing all scenarios, `combined_df` now contains all the data
print(combined_df)

               factor1  factor0.8  factor0.6  factor0.4  factor0.2  factor0
year                                                                       
2025      0.0      0.0        0.0        0.0        0.0        0.0      0.0
2026  13501.5  13501.5    13668.9    13839.7    14018.2    14196.6  14375.3
2027  13501.5  13501.5    13668.9    13839.7    14018.2    14196.6  14375.3
2028  23273.9  23273.9    23577.4    23883.7    24207.3    24530.9  24854.9
2029  23273.9  23273.9    23577.4    23883.7    24207.3    24530.9  24854.9


In [27]:
# Initialize the Results object
res_adjusted = Results("../data/outputs/03_example_CCTS_N-1_03_CT_infeasible_adjusted_scenarios")

flow = 'opex_yearly'
carrier = "co2_stored"

# Initialize an empty DataFrame to store the combined results
combined_df = pd.DataFrame()

# Iterate over each scenario
for scenario in res_adjusted.scenarios:
    # Export the flowdata to CSV for the current scenario
    res_adjusted.get_df(flow, scenario=scenario, to_csv="flow_adjusted.csv")
    
    # Load the exported data
    df = pd.read_csv('flow_adjusted.csv')
    
    # Find unique failure states and time steps
    time_steps = df['time_operation'].unique()
    
    # Filter for carrier
    df = df[df['carrier'] == carrier]
    
    # Initialize a list to store data for the current scenario
    scenario_data = []

    # Iterate over time_steps
    for time_step in time_steps:
        # Filter data for the current time_step
        df_time_step = df[df['time_operation'] == time_step]
        # Sum up the flow where there's no failure
        df_no_failure = df_time_step[df_time_step['failure_states'] == "no_failure_technology: no_failure_location"][flow].sum()
        # difference in flow for carrier fore each failure state minus no failure
        flow_difference = (df_time_step[flow].sum() - len(df['failure_states'].unique()) * df_no_failure) * 8.76
        total_flow = (df_time_step[flow].sum() - (len(df['failure_states'].unique())-1) * df_no_failure) * 8.76
        
        # Append the result to the list
        scenario_data.append(total_flow)
    
    # Create a DataFrame from the collected scenario data
    table_df = pd.DataFrame({"year": time_steps, scenario: scenario_data})
    
    # Adjust year based on your setup
    table_df['year'] = table_df['year'] + 2025
    
    if combined_df.empty:
        combined_df = table_df.set_index('year')
    else:
        # Merge the new scenario data into the combined DataFrame
        combined_df = combined_df.merge(table_df.set_index('year'), left_index=True, right_index=True, how='outer')

# After processing all scenarios, `combined_df` now contains all the data
print(combined_df)

KeyError: 'time_operation'

In [32]:
res_adjusted = Results("../data/outputs/03_example_CCTS_N-1_03_CT_infeasible_adjusted_scenarios")

flow = 'cost_total'
carrier = "co2_stored"

# Initialize an empty DataFrame to store the combined results
combined_df = pd.DataFrame()

# Iterate over each scenario
for scenario in res_adjusted.scenarios:
    # Export the flowdata to CSV for the current scenario
    res_adjusted.get_df(flow, scenario=scenario, to_csv="flow_adjusted.csv")
    
    # Load the exported data
    df = pd.read_csv('flow_adjusted.csv')
    
    # Find unique failure states and time steps
    print(df)

   year    cost_total
0     0  0.000000e+00
1     1  1.350149e+07
2     2  1.350149e+07
3     3  2.327387e+07
4     4  2.327387e+07
   year    cost_total
0     0  0.000000e+00
1     1  1.350149e+07
2     2  1.350149e+07
3     3  2.327387e+07
4     4  2.327387e+07
   year    cost_total
0     0  0.000000e+00
1     1  1.366887e+07
2     2  1.366887e+07
3     3  2.357740e+07
4     4  2.357740e+07
   year    cost_total
0     0  0.000000e+00
1     1  1.383973e+07
2     2  1.383973e+07
3     3  2.388373e+07
4     4  2.388373e+07
   year    cost_total
0     0  0.000000e+00
1     1  1.401817e+07
2     2  1.401817e+07
3     3  2.420732e+07
4     4  2.420732e+07
   year    cost_total
0     0  0.000000e+00
1     1  1.419661e+07
2     2  1.419661e+07
3     3  2.453091e+07
4     4  2.453091e+07
   year    cost_total
0     0  0.000000e+00
1     1  1.437526e+07
2     2  1.437526e+07
3     3  2.485487e+07
4     4  2.485487e+07


In [34]:
combined_df

""


In [139]:
res_adjusted.get_df('flow_conversion_output', scenario=scenario
                    )


technology         carrier      node              failure_states                              time_operation
capture_amine      co2_ambient  swiss_storage_CH  no_failure_technology: no_failure_location  0                 0.0
                                                                                              1                 0.0
                                                                                              2                 0.0
                                                                                              3                 0.0
                                                                                              4                 0.0
                                                                                                               ... 
permanent_storage  co2_stored   waste_2_CH        flowline: waste_2_CH-swiss_storage_CH       0                 0.0
                                                                               